In [187]:
%pip install lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
import xgboost as xgb
import lightgbm as lgb
import sklearn # For version check


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [188]:
df = pd.read_csv(r'C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\train.csv')
df

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,16.868428,NaN,93.53031757838667,14.393967,3.0,0.738911,12.147711,3.005355,26.206810,1.733013,12.594122273332914,1018.3744670739436,B2,E02,tracking,0.664907
19996,19996,53.415061,296.970303,93.98571391279083,25.997012,2.0,0.513061,0.000000,0.532119,65.000000,64.558667,0.9769909288128159,1016.081102065643,D4,E00,fixed,0.354070
19997,19997,2.442727,660.328019,37.9689180401391,32.818396,9.0,0.548602,13.047950,4.075498,11.584869,57.730134,4.750937249871706,1009.6844614602336,D4,NaN,tracking,0.419734
19998,19998,NaN,632.760700,43.01470184078199,19.063517,4.0,NaN,0.000000,1.068906,21.149351,78.123689,11.304158443374758,1006.6738746072241,A1,E00,tracking,0.661963


In [189]:
df.shape

(20000, 17)

In [190]:
df.size

340000

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  object 
 16  effi

In [192]:
df.describe().transpose() # this will select only the numeric rows

,count,mean,std,min,25%,50%,75%,max
id,20000.0,9999.500000,5773.647028,0.000000,4999.750000,9999.500000,14999.250000,19999.000000
temperature,18999.0,25.077241,12.513129,0.000000,16.853522,24.720345,32.848917,147.394168
irradiance,19013.0,501.273896,250.926590,-597.278646,332.227277,499.654730,668.416734,1537.810349
panel_age,18989.0,17.509758,10.097557,0.001264,8.777905,17.497731,26.340761,34.998379
maintenance_count,18973.0,4.012070,2.002268,0.000000,3.000000,4.000000,5.000000,15.000000
soiling_ratio,18990.0,0.698879,0.172244,0.400149,0.550654,0.697663,0.847838,0.999949
voltage,19007.0,16.242251,17.889031,0.000000,0.000000,12.350138,26.557322,494.279016
current,19023.0,1.713396,1.152953,0.000054,0.772311,1.558413,2.474744,7.315597
module_temperature,19022.0,29.923807,12.125405,0.000000,21.522124,29.857669,38.094943,65.000000
cloud_coverage,18990.0,51.378575,48.473664,0.000244,25.081618,49.704133,75.052824,1000.000000


In [193]:
df[df['efficiency'] == df['efficiency'].max()]

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
1616,1616,30.160975,1228.561162,26.450265215585077,3.988241,10.0,0.943199,17.583248,4.842217,29.477682,67.952173,8.36247200704732,1016.4608719691104,B2,NaN,fixed,0.987066


In [194]:
df['wind_speed'].value_counts()


wind_speed
badval                42
error                 41
unknown               36
1.4004639058482033     1
4.676583637669918      1
                      ..
9.196479438626795      1
0.7985304351521505     1
7.767053830591635      1
3.0572137101487558     1
8.321732514520054      1
Name: count, Length: 19884, dtype: int64

In [195]:
df['pressure'].value_counts()

pressure
unknown               46
error                 45
badval                44
1012.8003752419361     1
1001.3287852158609     1
                      ..
1021.4074573669117     1
1008.7504815771696     1
1014.7401210245107     1
1006.4182950874194     1
1030.4971803983756     1
Name: count, Length: 19868, dtype: int64

In [196]:
df.loc[df['pressure'].isin(['badval','error', 'unknown'])]

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
40,40,47.973606,NaN,22.058748668559303,16.093618,4.0,0.428969,47.819612,3.264913,52.238395,80.213642,2.7854029076788285,unknown,B2,E01,tracking,0.515999
79,79,1.891977,422.663365,36.660789342772034,11.312273,2.0,0.483908,2.548296,0.479120,7.818356,14.493848,7.80719789753973,unknown,D4,NaN,fixed,0.484963
94,94,40.368922,78.166392,83.62935788403288,4.336180,6.0,0.841042,28.894706,0.070557,45.190447,76.587267,0.7230370811744813,unknown,D4,NaN,fixed,0.435985
187,187,12.942382,193.882742,14.825030046292975,23.979997,6.0,0.680431,0.000000,1.349382,18.503371,29.296724,2.5304860869622026,unknown,C3,E00,tracking,0.412498
228,228,26.726921,286.689162,99.433526840726,22.590687,4.0,0.735756,NaN,0.555327,30.436082,52.092448,11.216905103677677,badval,B2,E02,fixed,0.395336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19505,19505,37.167240,558.010651,6.431461610112487,24.904775,3.0,0.511923,10.350710,1.308435,38.203510,NaN,4.274765098164273,badval,D4,E00,NaN,0.441483
19543,19543,25.404976,481.071234,22.801095249618697,15.689368,0.0,0.416402,7.713185,2.775103,36.081079,81.419422,12.128268170331731,unknown,A1,E00,dual-axis,0.460354
19626,19626,39.784297,669.728364,27.975323166237022,32.703050,5.0,0.737207,70.823170,0.681198,41.566154,52.773546,8.494866395022155,error,B2,E02,NaN,0.542955
19760,19760,13.751447,483.938795,74.28739629809242,14.937022,4.0,0.588709,19.464720,1.905392,18.353161,23.103253,7.7107782580320094,badval,A1,E02,NaN,0.469591


In [197]:
df['temperature'].value_counts()

temperature
0.000000      389
60.000000      32
100.000000      2
25.311591       1
7.817315        1
             ... 
28.091359       1
37.652488       1
7.981089        1
21.097337       1
53.339567       1
Name: count, Length: 18579, dtype: int64

Maximum temperatures are found as zero and the max temperature is above 100

In [198]:
df['irradiance'].value_counts()

irradiance
576.179270    1
240.003973    1
687.612799    1
735.141179    1
12.241203     1
             ..
258.801247    1
296.970303    1
660.328019    1
632.760700    1
793.745224    1
Name: count, Length: 19013, dtype: int64

In [199]:
df['irradiance'].isna().sum()

np.int64(987)

In [200]:
df['panel_age'].value_counts()

panel_age
32.135501    1
19.977460    1
1.496401     1
18.491582    1
30.722697    1
            ..
14.393967    1
25.997012    1
32.818396    1
19.063517    1
20.073134    1
Name: count, Length: 18989, dtype: int64

In [201]:
df['panel_age'].isna().sum()

np.int64(1011)

In [202]:
df['maintenance_count'].value_counts()

maintenance_count
3.0     3744
4.0     3703
5.0     2988
2.0     2785
6.0     1948
1.0     1331
7.0     1108
8.0      593
0.0      349
9.0      271
10.0      98
11.0      37
12.0      14
13.0       3
15.0       1
Name: count, dtype: int64

In [203]:
df['maintenance_count'].isna().sum()

np.int64(1027)

In [204]:
df['soiling_ratio'].value_counts()

soiling_ratio
0.803199    1
0.479456    1
0.822398    1
0.837529    1
0.551833    1
           ..
0.660319    1
0.738911    1
0.513061    1
0.548602    1
0.971922    1
Name: count, Length: 18990, dtype: int64

In [205]:
df['soiling_ratio'].isna().sum()

np.int64(1010)

In [206]:
df['voltage'].value_counts()

voltage
0.000000     5158
0.706348        1
13.228523       1
19.976573       1
1.670913        1
             ... 
12.297742       1
18.418306       1
52.207393       1
12.936880       1
5.356320        1
Name: count, Length: 13850, dtype: int64

In [207]:
df['voltage'].isna().sum()

np.int64(993)

Majority of Voltage are found to be 0. And I think voltage and current are the outputs from the solar panel so we can drop those columns because it wont affect the efficiency right?


In [208]:
df['current'].value_counts()

current
1.963787    1
0.241473    1
4.191800    1
0.960567    1
0.898062    1
           ..
3.005355    1
0.532119    1
4.075498    1
1.068906    1
4.017684    1
Name: count, Length: 19023, dtype: int64

In [209]:
df['current'].isna().sum()

np.int64(977)

In [210]:
unique_voltage_not_in_current = df.loc[~df['voltage'].isin(df['current']),'voltage'].unique()
unique_voltage_not_in_current


array([37.40352671, 21.84331546, 48.22288151, ..., 12.14771133,
       13.04795014, 41.24761567], shape=(13850,))

In [211]:
df['module_temperature'].value_counts()

module_temperature
65.000000    40
0.000000     20
43.363708     1
57.720436     1
11.426214     1
             ..
37.571833     1
38.674178     1
12.837497     1
28.609224     1
27.545096     1
Name: count, Length: 18964, dtype: int64

In [212]:
df.loc[df['module_temperature']> 60]


,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
115,115,55.127610,367.626823,12.025989217410249,6.477015,2.0,0.732385,0.000000,0.899403,60.196444,99.678876,13.45143154498761,1019.0676713780514,B2,E01,dual-axis,0.503151
189,189,57.830386,912.731983,86.17867687320704,12.091633,2.0,0.797005,39.061541,2.168671,65.000000,13.777121,5.216506810295259,1007.7362253936187,D4,NaN,NaN,0.694131
505,505,51.878531,364.126123,32.55784474251867,26.825775,5.0,NaN,0.000000,0.382291,61.284818,40.602980,4.690653945495291,1012.8109991728885,A1,E01,dual-axis,0.536272
697,697,59.843856,259.090389,64.39577240738956,25.630764,6.0,0.414344,20.919495,2.156268,62.705516,89.180627,5.438549240729484,1033.2915018578383,C3,NaN,fixed,0.377735
786,786,59.119279,511.053095,18.761205627512755,31.711366,6.0,0.415809,7.961602,0.944160,65.000000,94.702918,8.057307435405985,1011.9842728791381,C3,E01,fixed,0.423003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,19515,56.315526,672.233633,78.8767547865067,4.680563,3.0,0.883854,21.680085,1.175329,64.262647,3.649416,5.297399374294098,1016.0428011511001,C3,E02,NaN,0.666963
19553,19553,60.000000,699.522674,17.564033514055122,0.909439,4.0,0.953168,4.100672,1.763171,64.968603,59.394887,5.093627444442655,1032.1133473843047,A1,NaN,fixed,0.716516
19783,19783,52.108308,608.764072,70.696599430603,21.050955,2.0,0.627764,38.452939,2.052685,60.917099,63.678424,0.26185323931255666,1029.8303343875616,A1,E01,dual-axis,0.471451
19889,19889,60.000000,325.551636,29.425686018864393,18.930643,5.0,0.629743,4.294408,2.554062,65.000000,62.149085,3.0790402972870163,1002.3878716464516,C3,E01,NaN,0.449122


In [213]:
df['module_temperature'].isna().sum()

np.int64(978)

In [214]:
df['cloud_coverage'].value_counts()

cloud_coverage
1000.000000    32
32.113179       1
75.405076       1
77.078146       1
65.969637       1
               ..
75.307073       1
93.401635       1
4.587801        1
42.703317       1
54.483190       1
Name: count, Length: 18959, dtype: int64

In [215]:
df['cloud_coverage'].isna().sum()

np.int64(1010)

In [216]:
df['wind_speed'].value_counts()

wind_speed
badval                42
error                 41
unknown               36
1.4004639058482033     1
4.676583637669918      1
                      ..
9.196479438626795      1
0.7985304351521505     1
7.767053830591635      1
3.0572137101487558     1
8.321732514520054      1
Name: count, Length: 19884, dtype: int64

In [217]:
df['wind_speed'].isna().sum()

np.int64(0)

In [218]:
df['installation_type'].value_counts()

installation_type
tracking     5067
fixed        4990
dual-axis    4915
Name: count, dtype: int64

In [219]:
df['installation_type'].isna().sum()

np.int64(5028)

In [220]:
train_df = pd.read_csv(r'C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\train.csv')
test_df = pd.read_csv(r"C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\test.csv")
sample_submission_df = pd.read_csv(r"C:\Users\kumara\Downloads\1a513221-3-dataset\dataset\sample_submission.csv")

In [221]:
test_ids = test_df['id']
y_train_full = train_df['efficiency']
train_df_features = train_df.drop('efficiency', axis=1)

In [222]:
combined_df = pd.concat([train_df_features, test_df], ignore_index=True)
train_len = len(train_df_features)

In [223]:
combined_df['pressure'] = combined_df['pressure'].replace('badval', np.nan)
combined_df['humidity'] = combined_df['humidity'].replace('unknown', np.nan)

In [224]:
cols_to_numeric = ['temperature', 'irradiance', 'humidity', 'panel_age',
                   'maintenance_count', 'soiling_ratio', 'voltage', 'current',
                   'module_temperature', 'cloud_coverage', 'wind_speed', 'pressure']
for col in cols_to_numeric:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20000 non-null  int64  
 1   temperature         18999 non-null  float64
 2   irradiance          19013 non-null  float64
 3   humidity            20000 non-null  object 
 4   panel_age           18989 non-null  float64
 5   maintenance_count   18973 non-null  float64
 6   soiling_ratio       18990 non-null  float64
 7   voltage             19007 non-null  float64
 8   current             19023 non-null  float64
 9   module_temperature  19022 non-null  float64
 10  cloud_coverage      18990 non-null  float64
 11  wind_speed          20000 non-null  object 
 12  pressure            20000 non-null  object 
 13  string_id           20000 non-null  object 
 14  error_code          14088 non-null  object 
 15  installation_type   14972 non-null  object 
 16  effi

In [226]:
new_df = combined_df.drop(columns=['voltage','current','string_id','error_code','installation_type'], axis= 1)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  32000 non-null  int64  
 1   temperature         30417 non-null  float64
 2   irradiance          30398 non-null  float64
 3   humidity            31800 non-null  float64
 4   panel_age           30382 non-null  float64
 5   maintenance_count   30364 non-null  float64
 6   soiling_ratio       30380 non-null  float64
 7   module_temperature  30442 non-null  float64
 8   cloud_coverage      30408 non-null  float64
 9   wind_speed          31800 non-null  float64
 10  pressure            31800 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 2.7 MB


In [229]:
numerical_features = combined_df.select_dtypes(include=np.number).columns.tolist()
if 'id' in numerical_features:
    numerical_features.remove('id')
categorical_features = combined_df.select_dtypes(include='object').columns.tolist()

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='drop'
)

